# Queue Model (M/M/c/K) Analysis with PyDASA

This notebook demonstrates **dimensional analysis** of queueing systems using PyDASA's workflows with a custom dimensional framework. We'll explore how to:

1. Define a custom dimensional framework (Time, Structure, Data)
2. Model software service variables with custom dimensions
3. Use PyDASA's `AnalysisEngine` to derive dimensionless groups
4. Understand the M/M/c/K queue model and operational metrics
5. Perform sensitivity analysis and Monte Carlo simulation
6. Visualize the "Yoly" trade-off chart for system design

## What is the M/M/c/K Queue Model?

The **M/M/c/K** queue model is a fundamental queueing system in performance analysis:

- **M** (Markovian arrivals): Request arrivals follow a Poisson process with rate $\lambda$
- **M** (Markovian service): Service times follow exponential distribution with rate $\mu$
- **c**: Number of parallel servers (resources) available
- **K**: Maximum system capacity (queue + servers)

### Key Performance Metrics:

**Average Waiting Time** ($W$):
$$
W = f(\lambda, \mu, c, K, L, ...)
$$

**Traffic Intensity** ($\tau$):
$$
\tau = \frac{\lambda}{\mu}
$$

**System Utilization** ($\rho$):
$$
\rho = \frac{\lambda}{c \cdot \mu}
$$

### Error Rate and Effective Response

The system has an error rate ($err$) that affects the effective response rate:
$$
\chi = (1 - err) \cdot \lambda
$$

Where:
- $\chi$ (chi): Effective response rate accounting for errors [S/T]
- $err$: Error/failure rate [dimensionless, 0 to 1]
- $\lambda$: Arrival rate [S/T]

**Example**: If $\lambda = 100$ req/s and $err = 0.02$ (2% error rate), then $\chi = 0.98 \times 100 = 98$ req/s successfully served.

### Custom Dimensional Framework (T, S, D)

For software service analysis, we introduce three fundamental dimensions:
- **T** (Time): Temporal measurements [seconds]
- **S** (Structure): Capacity, servers, queue slots [units]
- **D** (Data): Information content, memory [bytes]

### The "Yoly" Concept

**Yoly** is a composite happiness metric that captures the trade-off between:
- **Performance**: Fast response times (low $W$)
- **Availability**: Low utilization (prevents saturation)
- **Memory Efficiency**: Optimal buffer allocation
- **Reliability**: Low error rates (high $\chi/\lambda$ ratio)

A high Yoly score indicates a well-balanced system configuration that keeps users happy!

## 1. Import Required Libraries

First, let's import PyDASA's core modules for dimensional analysis with custom frameworks.

In [ ]:
# PyDASA imports
import pydasa
from pydasa.workflows.phenomena import AnalysisEngine
from pydasa.elements.parameter import Variable
from pydasa.dimensional.vaschy import Schema

# For visualization and analysis
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatterSciNotation
import random


print(f"VERSION: {pydasa.__version__}")
print("PyDASA imported successfully!")

VERSION: 0.6.8
	PyDASA imported successfully!


## 2. Define Custom Dimensional Framework

For software service analysis, we need a custom dimensional framework that captures the unique characteristics of queueing systems. Unlike physical systems (Length, Mass, Time), we define the following fundamental dimensional units:

**Custom Framework Definition:**

1. **T (Time)**: Temporal measurements
   - Unit: seconds [s].
   - Examples: waiting time, service time, inter-arrival time.
   - Physical analogy: Similar to Time in PHYSICAL framework.

2. **S (Structure)**: System capacity and architectural elements
   - Unit: requests [req].
   - Examples: number of servers, queue positions, concurrent requests.
   - Software-specific: Represents discrete structural resources.

3. **D (Data)**: Information content and memory.
   - Unit: bit [bit].
   - Examples: request payload size, buffer memory, data throughput.
   - Software-specific: Quantifies information processing.

This framework allows us to perform dimensional analysis on software services, deriving dimensionless numbers that characterize system behavior independent of scale.

In [ ]:
# Define custom Fundamental Dimensional Units (FDUs) for software service analysis
fdu_list = [
    {
        "_idx": 0,
        "_sym": "T",
        "_fwk": "CUSTOM",
        "description": "Temporal measurements",
        "_unit": "s",
        "_name": "Time"
    },
    {
        "_idx": 1,
        "_sym": "S",
        "_fwk": "CUSTOM",
        "description": "System capacity and structural resources",
        "_unit": "units",
        "_name": "Structure"
    },
    {
        "_idx": 2,
        "_sym": "D",
        "_fwk": "CUSTOM",
        "description": "Information content and memory",
        "_unit": "bytes",
        "_name": "Data"
    }
]

# Create custom schema with T, S, D framework
schema = Schema(_fwk="CUSTOM",
                _fdu_lt=fdu_list, _idx=0)   # type: ignore
schema._setup_fdus()

print("=== Custom Framework Created Successfully! ===")
print(f"\tFramework: {schema.fwk}")
print(f"\tNumber of FDUs: {len(schema._fdu_lt)}")
print("\nFundamental Dimensional Units:")
print(f"{'Symbol':<10} {'Name':<15} {'Unit':<10} {'Description':<55}")
print("-" * 80)
for fdu in schema._fdu_lt:
    print(f"{fdu._sym:<10} {fdu._name:<15} {fdu._unit:<10} {fdu.description[:60]}")

=== Custom Framework (T, S, D) Created Successfully! ===
	Framework: CUSTOM
	Number of FDUs: 3

Fundamental Dimensional Units:
Symbol     Name            Unit       Description                                            
--------------------------------------------------------------------------------
T          Time            s          Temporal measurements
S          Structure       units      System capacity and structural resources
D          Data            bytes      Information content and memory


## 3. Define Queue Model Variables

We'll define 10 variables for the M/M/c/K queueing system with proper dimensions using the custom T, S, D framework.

**Variable Categories:**

**INPUT Variables (3)** - Primary system parameters:
1. **λ** (lambda): Arrival rate [S/T]
2. **K**: Maximum queue capacity [S]
3. **ρ_req**: Data density per request [D/S]

**OUTPUT Variable (1)** - Performance metric we want to analyze:
1. **W**: Average waiting time [T]

**CONTROL Variables (6)** - System configuration and secondary parameters:
1. **L**: Average queue length [S]
2. **μ** (mu): Service rate [S/T]
3. **c**: Number of parallel servers [S]
4. **B**: Buffer allocated memory [D]
5. **err**: Error/failure rate [dimensionless]
6. **χ** (chi): Effective response rate [S/T], where χ = (1 - err) × λ

**Typical Values for a Web Service:**
- Arrival rate: 100 req/s
- Service rate: 120 req/s (slightly faster than arrivals)
- Servers: 4 parallel processors
- Queue capacity: 1000 requests
- Buffer memory: 1 GB
- Data per request: 1 KB
- Error rate: 2% (0.02)
- Average queue length: 200 requests
- Waiting time: 50 ms

In [24]:
# Define the queueing system variables as dictionaries
# Each variable is stored with its symbol as the key

variables_dict = {
    # INPUT VARIABLES (3)
    # Arrival rate: λ [S/T] - requests arriving per unit time
    "\\lambda_{s}": {
        "_idx": 0,
        "_sym": "\\lambda_{s}",
        "_alias": "lambda_s",
        "_fwk": "CUSTOM",
        "_cat": "IN",
        "_name": "Arrival Rate",
        "description": "Request arrival rate (Poisson process)",
        "relevant": True,
        "_dims": "S*T^-1",
        "_units": "req/s",
        "_setpoint": 200.0,     # req/s
        "_std_setpoint": 200.0,
        "_std_mean": 200.0,
        "_std_min": 100.0,
        "_std_max": 500.0,
        "_step": 10.0,
    },

    # Queue capacity: K [S] - maximum system capacity
    "K": {
        "_idx": 1,
        "_sym": "K",
        "_alias": "K",
        "_fwk": "CUSTOM",
        "_cat": "IN",
        "_name": "Queue Capacity",
        "description": "Maximum system capacity (queue + servers)",
        "relevant": True,
        "_dims": "S",
        "_units": "requests",
        "_setpoint": 10.0,      # requests
        "_std_setpoint": 10.0,
        "_std_mean": 10.0,
        "_std_min": 8.0,
        "_std_max": 12.0,
        "_step": 1.0,
    },
    # Data density: ρ_req [D/S] - data per request
    "\\rho_{req}": {
        "_idx": 2,
        "_sym": "\\rho_{req}",
        "_alias": "rho_req",
        "_fwk": "CUSTOM",
        "_cat": "IN",
        "_name": "Data Density",
        "description": "Data content per request",
        "relevant": True,
        "_dims": "D*S^-1",
        "_units": "MB/req",
        "_setpoint": 1.0,           # 1 MB per request
        "_min": 0.001,              # 1 kB per request
        "_max": 10.0,               # 10 MB per request
        "_std_units": "bit/req",
        "_std_setpoint": 8000000,
        "_std_mean": 8000000.0,
        "_std_min": 8000,
        "_std_max": 80000000,
        "_step": 5*8000,           # 5 kbit steps in std units
    },

    # OUTPUT VARIABLE (1)
    # Queue length: L [S] - average number in queue
    "L": {
        "_idx": 4,
        "_sym": "L",
        "_alias": "L",
        "_fwk": "CUSTOM",
        "_cat": "OUT",
        "_name": "Queue Length",
        "description": "Average queue length",
        "relevant": True,
        "_dims": "S",
        "_units": "requests",
        "_setpoint": 0.9946,        # req, depends on M/M/c/K parameters
        "_std_setpoint": 0.9946,
        "_std_min": 0.9946,
        "_std_max": 4.9750,
        "_std_mean": 0.9946,
    },
    
    # Waiting time: W [T] - average time in queue
    "W": {
        "_idx": 3,
        "_sym": "W",
        "_alias": "W",
        "_fwk": "CUSTOM",
        "_cat": "CTRL",
        "_name": "Waiting Time",
        "description": "Average waiting time in queue",
        "relevant": True,
        "_dims": "T",
        "_units": "ms",
        "_setpoint": 5.0,         # waiting time, depends on M/M/c/K parameters
        "_std_min": 5.0,
        "_std_max": 13.7,
        "_std_units": "s",
        "_std_setpoint": 0.0050,
        "_std_mean": 0.0050,
        "_std_min": 0.0050,
        "_std_max": 0.0137,
    },

    # Service rate: μ [S/T] - requests served per unit time
    "\\mu_{s}": {
        "_idx": 5,
        "_sym": "\\mu_{s}",
        "_alias": "mu_comp",
        "_fwk": "CUSTOM",
        "_cat": "CTRL",
        "_name": "Service Rate",
        "description": "Service completion rate per server",
        "relevant": True,
        "_dims": "S*T^-1",
        "_units": "req/s",
        "_setpoint": 400.0,        # req/s, constant
        "_std_units": "req/s",
        "_std_setpoint": 400.0,
        "_std_mean": 400.0,
        "_std_min": 400.0,
        "_std_max": 500.0,
    },

    # Response rate: χ [S/T] - effective throughput
    "\\chi_{s}": {
        "_idx": 9,
        "_sym": "\\chi_{s}",
        "_alias": "chi_comp",
        "_fwk": "CUSTOM",
        "_cat": "CTRL",
        "_name": "Response Rate",
        "description": "Effective response rate: χ = (1 - err) × λ",
        "relevant": True,
        "_dims": "S*T^-1",
        "_units": "req/s",
        "_setpoint": 97.0,         # (1 - 0.03) × 100 = 97 req/s, depends on error rate and arrival rate
        "_std_units": "req/s",
        "_std_setpoint": 97.0,
        "_std_min": 97.0,          # min: (1 - 0.03) × 100
        "_std_max": 485.0,         # max: (1 - 0.03) × 500
    },

    # Servers: c [S] - number of parallel servers
    "c": {
        "_idx": 6,
        "_sym": "c",
        "_alias": "c",
        "_fwk": "CUSTOM",
        "_cat": "CTRL",
        "_name": "Servers",
        "description": "Number of parallel requests handled by servers/processors",
        "relevant": True,
        "_dims": "S",
        "_units": "req",
        "_setpoint": 1.0,           # 1 request (1 server)
        "_std_units": "req",
        "_std_setpoint": 1.0,
        "_std_min": 1.0,
        "_std_max": 4.0,            # up to 8 servers (for scaling analysis)
        "_std_mean": 2.0,
    },

    # Buffer memory: B [D] - allocated memory
    "B": {
        "_idx": 7,
        "_sym": "B",
        "_alias": "B",
        "_fwk": "CUSTOM",
        "_cat": "CTRL",
        "_name": "Buffer Memory",
        "description": "Total allocated buffer memory",
        "relevant": True,
        "_dims": "D",
        "_units": "MB",
        "_setpoint": 16,            # 16 MB
        "_min": 16,
        "_max": 32,
        "_std_units": "bit",
        "_std_setpoint": 1.28e8,    # 128 Mb
        "_std_mean": 1.28e8,    # 128 Mb
        "_std_min": 1.28e8,         # 128 Mb
        "_std_max": 2.56e8,         # 256 Mb, (for scaling analysis)
    },

    # Error rate: err [dimensionless] - failure probability
    # NOT relevant for dimensional analysis (just a coefficient)
    "err": {
        "_idx": 8,
        "_sym": "err",
        "_alias": "err",
        "_fwk": "CUSTOM",
        "_cat": "CTRL",
        "_name": "Error Rate",
        "description": "Request error/failure rate (0 to 1)",
        "relevant": False,         # NOT part of dimensional matrix
        "_dims": "n.a.",               # Dimensionless
        "_units": "%",
        "_setpoint": 0.03,         # 3% error rate, constant
        "_std_units": "",
        "_std_setpoint": 0.03,
        "_std_min": 0.01,
        "_std_max": 0.04,
        "_std_mean": 0.03,
    },
}

# Q(M/M/c/K) queueing system variables:
# - λ: Arrival rate (S/T) = 200
# - K: Queue capacity (S) = 10
# - c: Servers (S) = 1
# - μ: Service rate (S/T) = 400
# - ρ_req: Data density (D/S) = 1 MB/req
# - B: Buffer memory (D) = 16 MB
# - err: Error rate (dimensionless) = 0.03 (3% failure rate)
# - χ: Response rate (S/T) = 97 req/s (calculated as (1 - err) × λ)
# Outputs (calculated from M/M/c/K formulas and error rate):
# - W: Waiting time (T) = 0.005 s
# - L: Queue length (S) = 0.9946 req
# - Wq: Waiting time in queue (T) = 0.0025 s
# - Lq: Queue length in queue (S) = 0.4949 req

# Convert dictionaries to Variable instances
variables = {
    sym: Variable(**params) for sym, params in variables_dict.items()
}

print("=== Variables defined ===")
print(f"\t- Total: {len(variables)}")
print(f"\t- Relevant for dimensional analysis: {sum(1 for v in variables.values() if v.relevant)}")
print(f"\t- Not relevant variables: {sum(1 for v in variables.values() if not v.relevant)}")

print("\nVariable Details:")
print(f"{'Symbol':<15} {'Setpoint':<20} {'Units':<10} {'Dimensions':<20} {'Category':<10} {'Relevant':<10}")
print("-" * 95)
for sym, var in variables.items():
    dims_str = var.dims if var.dims else "(N.A.)"
    relevant_str = "✓" if var.relevant else "✗"
    print(f"{sym:<15} {var.setpoint:<20} {var.units:<10} {dims_str:<20} {var.cat:<10} {relevant_str:<10}")

=== Variables defined ===
	- Total: 10
	- Relevant for dimensional analysis: 9
	- Not relevant variables: 1

Variable Details:
Symbol          Setpoint             Units      Dimensions           Category   Relevant  
-----------------------------------------------------------------------------------------------
\lambda_{s}     200.0                req/s      S*T^-1               IN         ✓         
K               10.0                 requests   S                    IN         ✓         
\rho_{req}      1.0                  MB/req     D*S^-1               IN         ✓         
L               0.9946               requests   S                    OUT        ✓         
W               5.0                  ms         T                    CTRL       ✓         
\mu_{s}         400.0                req/s      S*T^-1               CTRL       ✓         
\chi_{s}        97.0                 req/s      S*T^-1               CTRL       ✓         
c               1.0                  req        S

## 3. Create Dimensional Analysis Engine

Now we'll use PyDASA's **AnalysisEngine** (main workflow) to automatically derive dimensionless groups using the Buckingham Pi theorem.

In [ ]:
# Create the dimensional analysis engine
engine = AnalysisEngine(
    _idx=0,
    _fwk="CUSTOM",
    _schema=schema,
    _name="Software Service Analysis",
    description="Dimensional analysis for the Yoly Chart and the queue model M/M/c/K."
)

# Add all variables to the engine
engine.variables = variables

print("=== AnalysisEngine Created ===")
print(f"\t- Framework: {engine.fwk}")
print(f"\t- Variables: {len(variables)}\n")

print("Variable Categories:")
print("-" * 50)
n_inputs = sum(1 for v in variables.values() if v.cat == "IN")
print(f"\t- Input variables: {n_inputs}")
n_outputs = sum(1 for v in variables.values() if v.cat == "OUT")
print(f"\t- Output variables: {n_outputs}")
n_controls = sum(1 for v in variables.values() if v.cat == "CTRL")
print(f"\t- Control variables: {n_controls}")

=== AnalysisEngine Created ===
	- Framework: CUSTOM
	- Variables: 10

Variable Categories:
--------------------------------------------------
	- Input variables: 3
	- Output variables: 1
	- Control variables: 6


## 4. Run Dimensional Analysis

Execute the complete workflow to generate dimensionless coefficients (Pi groups).

In [39]:
# Run the complete dimensional analysis workflow
results = engine.run_analysis()

print(f"=== Analysis complete! ===")
print(f"\tNumber of Dimensionless Groups: {len(engine.coefficients)}")
print(f"\tCoefficients generated: {list(engine.coefficients.keys())}\n")

# Display coefficients in a formatted table
print("Dimensionless Coefficients Details:")
print("=" * 100)
print(f"{'Coefficient':<15} {'Expression':<35} {'Variable Exponents':<60}")
print("=" * 100)

for name, coeff in engine.coefficients.items():
    # Extract the coefficient expression
    expression = str(coeff.pi_expr) if len(str(coeff.pi_expr)) < 35 else str(coeff.pi_expr)[:32] + "..."
    
    # Format variable exponents as a compact string
    exponents_str = ", ".join([f"{var}^{exp}" if exp != 1 else var
                               for var, exp in coeff.var_dims.items()])
    if len(exponents_str) > 58:
        exponents_str = exponents_str[:55] + "..."
    
    print(f"{name:<15} {expression:<35} {exponents_str:<60}")

print("=" * 100)

=== Analysis complete! ===
	Number of Dimensionless Groups: 6
	Coefficients generated: ['\\Pi_{0}', '\\Pi_{1}', '\\Pi_{2}', '\\Pi_{3}', '\\Pi_{4}', '\\Pi_{5}']

Dimensionless Coefficients Details:
Coefficient     Expression                          Variable Exponents                                          
\Pi_{0}         \frac{L}{K}                         K^-1, L                                                     
\Pi_{1}         \frac{\lambda_{s}*W}{K}             \lambda_{s}, K^-1, W                                        
\Pi_{2}         \frac{\mu_{s}}{\lambda_{s}}         \lambda_{s}^-1, \mu_{s}                                     
\Pi_{3}         \frac{\chi_{s}}{\lambda_{s}}        \lambda_{s}^-1, \chi_{s}                                    
\Pi_{4}         \frac{c}{K}                         K^-1, c                                                     
\Pi_{5}         \frac{B}{K*\rho_{req}}              K^-1, \rho_{req}^-1, B                                      


## 5. Derive Key Dimensionless Coefficients

Now let's use PyDASA's `derive_coefficient()` method to create operationally meaningful coefficients from the Pi groups. Thus, we expect to see:

1. **Occupancy Coefficient (σ)**: Queue capacity utilization - σ = Π₀ = L/K
2. **Stall Coefficient (δ)**: Service blocking indicator - δ = Π₁ = W·λ/L
3. **Efficiency Coefficient (η)**: Resource utilization effectiveness - η = Π₂⁻¹·Π₃·Π₄⁻¹ = χ·K/(μ·c)
4. **Payload Coefficient (φ)**: Data density metric - φ = Π₅ = B/(ρ_req·K)

These derived coefficients connect directly to the Yoly diagram we'll construct.

In [41]:
# Get the original Pi coefficients
pi_keys = list(engine.coefficients.keys())

# Derive Occupancy Coefficient: σ = Π₀ = L/K
sigma_coeff = engine.derive_coefficient(
    expr=f"{pi_keys[0]}",
    symbol="\\sigma",
    name="Occupancy Coefficient",
    description="σ = L/K - Queue occupancy ratio (0 = empty, 1 = full)",
    idx=-1
)

# Derive Stall Coefficient: δ = Π₁ = W·λ/L
delta_coeff = engine.derive_coefficient(
    expr=f"{pi_keys[1]}",
    symbol="\\delta",
    name="Stall Coefficient",
    description="δ = W·λ/L - Service stall/blocking indicator",
    idx=-1
)

# Derive Efficiency Coefficient: η = Π₂⁻¹·Π₃·Π₄⁻¹ = χ·K/(μ·c)
eta_coeff = engine.derive_coefficient(
    expr=f"{pi_keys[2]}**(-1) * {pi_keys[3]} * {pi_keys[4]}**(-1)",
    symbol="\\eta",
    name="Efficiency Coefficient",
    description="η = χ·K/(μ·c) - Resource utilization effectiveness",
    idx=-1
)

# Derive Payload Coefficient: φ = Π₅ = B/(ρ_req·K)
phi_coeff = engine.derive_coefficient(
    expr=f"{pi_keys[5]}",
    symbol="\\phi",
    name="Payload Coefficient",
    description="φ = B/(ρ_req·K) - Data density metric",
    idx=-1
)

# Calculate numerical values using stored setpoints
sigma_val = sigma_coeff.calculate_setpoint()
delta_val = delta_coeff.calculate_setpoint()
eta_val = eta_coeff.calculate_setpoint()
phi_val = phi_coeff.calculate_setpoint()

# Display results
print("=" * 120)
print("DERIVED DIMENSIONLESS COEFFICIENTS")
print("=" * 120)
print(f"{'Coefficient':<25} {'Expression':<45} {'Value':<15} {'Description':<35}")
print("=" * 120)
print(f"{'Occupancy (σ)':<25} {str(sigma_coeff.pi_expr):<45} {sigma_val:<15.4f} {'Queue capacity utilization':<35}")
print(f"{'Stall (δ)':<25} {str(delta_coeff.pi_expr)[:45]:<45} {delta_val:<15.4f} {'Service blocking indicator':<35}")
print(f"{'Efficiency (η)':<25} {str(eta_coeff.pi_expr)[:45]:<45} {eta_val:<15.4f} {'Resource utilization':<35}")
print(f"{'Payload (φ)':<25} {str(phi_coeff.pi_expr):<45} {phi_val:<15.4f} {'Data density metric':<35}")
print("=" * 120)

# Interpret values for operational insights
print("\nTRADITIONAL QUEUE OPERATIONAL ASSESSMENT")
print("=" * 120)

# traffic intensity: t = λ/μ
lambda_s = variables["\\lambda_{s}"].setpoint
mu_s = variables["\\mu_{s}"].setpoint
theta = lambda_s / mu_s    # type: ignore
print(f"Traffic Intensity (θ = λ/μ) = {theta:.2f}")

# utilization: ρ = λ/(c·μ)
c = variables["c"].setpoint
rho = lambda_s / (c* mu_s)    # type: ignore
print(f"Utilization (ρ = λ/(c·μ)) = {rho:.2f}")

# Occupancy L/K
L = variables["L"].setpoint
K = variables["K"].setpoint
occupancy = L / K    # type: ignore
print(f"Occupancy (L/K) = {occupancy:.4f}")

DERIVED DIMENSIONLESS COEFFICIENTS
Coefficient               Expression                                    Value           Description                        
Occupancy (σ)             \frac{L}{K}                                   0.0995          Queue capacity utilization         
Stall (δ)                 \frac{\lambda_{s}*W}{K}                       0.1000          Service blocking indicator         
Efficiency (η)            \frac{K*\chi_{s}}{\mu_{s}*c}                  2.4250          Resource utilization               
Payload (φ)               \frac{B}{K*\rho_{req}}                        1.6000          Data density metric                

TRADITIONAL QUEUE OPERATIONAL ASSESSMENT
Traffic Intensity (θ = λ/μ) = 0.50
Utilization (ρ = λ/(c·μ)) = 0.50
Occupancy (L/K) = 0.0995


## 7. Sensitivity Analysis

Let's use PyDASA's **SensitivityAnalysis** workflow to understand which variables have the most influence on our dimensionless coefficients.

In [42]:
from pydasa.workflows.influence import SensitivityAnalysis

# Create sensitivity analysis handler
sensitivity = SensitivityAnalysis(
    _idx=0,
    _fwk="CUSTOM",
    _schema=schema,
    _name="Queue Model Sensitivity Analysis",
    _cat="SYM"  # Symbolic sensitivity analysis
)

# Configure with variables and coefficients from the engine
sensitivity.variables = engine.variables
sensitivity.coefficients = engine.coefficients

print("=== SensitivityAnalysis workflow created ===")
print(f"\tFramework: {sensitivity.fwk}")
print(f"\tAnalysis type: {sensitivity.cat} (Symbolic)")
print(f"\tVariables: {len(sensitivity.variables)}")
print(f"\tCoefficients to analyze: {len(sensitivity.coefficients)}")

# Run symbolic sensitivity analysis at mean values
print("\n--- Running Symbolic Sensitivity Analysis ---")
sensitivity_results = sensitivity.analyze_symbolic(val_type="mean")

print(f"=== Symbolic sensitivity analysis complete! ===")
print(f"\tAnalyzed {len(sensitivity_results)} coefficients")
print(f"\tResults available for: {list(sensitivity_results.keys())}")

=== SensitivityAnalysis workflow created ===
	Framework: CUSTOM
	Analysis type: SYM (Symbolic)
	Variables: 10
	Coefficients to analyze: 10

--- Running Symbolic Sensitivity Analysis ---
=== Symbolic sensitivity analysis complete! ===
	Analyzed 10 coefficients
	Results available for: ['SEN_{\\Pi_{0}}', 'SEN_{\\Pi_{1}}', 'SEN_{\\Pi_{2}}', 'SEN_{\\Pi_{3}}', 'SEN_{\\Pi_{4}}', 'SEN_{\\Pi_{5}}', 'SEN_{\\sigma}', 'SEN_{\\delta}', 'SEN_{\\eta}', 'SEN_{\\phi}']


In [29]:
# Display sensitivity analysis results in formatted tables
print("=" * 120)
print("SENSITIVITY ANALYSIS RESULTS - Symbolic Differentiation at Mean Values")
print("=" * 120)

# Get the derived coefficients we care about (σ, δ, η, φ)
derived_coeff_keys = [
    k for k in sensitivity_results.keys() if not k.startswith("SEN_{\\Pi")
]

for coeff_key in derived_coeff_keys:
    sens_data = sensitivity_results[coeff_key]
    
    # Get coefficient name
    if coeff_key in engine.coefficients:
        coeff_name = engine.coefficients[coeff_key].name
    else:
        coeff_name = coeff_key
    
    print(f"\n{'Coefficient':<25} {coeff_key:<20} ({coeff_name})")
    print("-" * 120)
    print(f"{'Variable':<20} {'Sensitivity':<20} {'Relative Impact (%)':<25} {'Description':<55}")
    print("-" * 120)
    
    # Calculate total sensitivity for relative percentages
    total_sens = sum(abs(v) for v in sens_data.values() if isinstance(v, (int, float)))
    
    # Sort by absolute sensitivity (descending)
    sorted_vars = sorted(sens_data.items(), key=lambda x: abs(x[1]) if isinstance(x[1], (int, float)) else 0, reverse=True)
    
    for var_sym, sens_val in sorted_vars:
        if isinstance(sens_val, (int, float)):
            # Get variable name
            if var_sym in engine.variables:
                var_name = engine.variables[var_sym].name
            else:
                var_name = var_sym
            
            # Calculate relative impact
            rel_impact = (abs(sens_val) / total_sens * 100) if total_sens > 0 else 0
            
            # Format sensitivity value
            sens_str = f"{sens_val:+.4e}"
            
            # Impact description
            if rel_impact > 40:
                impact_desc = "🔴 DOMINANT influence"
            elif rel_impact > 20:
                impact_desc = "🟠 MAJOR influence"
            elif rel_impact > 10:
                impact_desc = "🟡 MODERATE influence"
            else:
                impact_desc = "🟢 MINOR influence"
            
            print(f"{var_sym:<20} {sens_str:<20} {rel_impact:<25.2f} {impact_desc:<55}")

print("\n" + "=" * 120)

SENSITIVITY ANALYSIS RESULTS - Symbolic Differentiation at Mean Values

Coefficient               SEN_{\sigma}         (SEN_{\sigma})
------------------------------------------------------------------------------------------------------------------------
Variable             Sensitivity          Relative Impact (%)       Description                                            
------------------------------------------------------------------------------------------------------------------------
L                    +1.0000e-01          90.95                     🔴 DOMINANT influence                                   
K                    -9.9460e-03          9.05                      🟢 MINOR influence                                      

Coefficient               SEN_{\delta}         (SEN_{\delta})
------------------------------------------------------------------------------------------------------------------------
Variable             Sensitivity          Relative Impact (%)       

## 7. Grid-Based Monte Carlo Data Generation

Instead of random sampling, we'll generate structured data points by systematically varying queue parameters:

**Strategy**: Grid search over configurations while varying arrival rate (λ) until near-saturation

**Configuration Parameters**:
- **err** (Error rate): [0.03, 0.01] → 2 values (3% and 1% failure rates)
- **μ** (Service rate): [400, 500] req/s → 2 values  
- **c** (Servers): [1, 2, 4] → 3 values
- **Total configurations**: 2 × 2 × 3 = **12 configurations**

**Constants**:
- **K** (Capacity) = 10 requests (fixed)
- **B** (Buffer) = 16 MB (fixed)
- **ρ_req** (Data density) = 1.0 MB/req (fixed)

**Lambda Sweep Strategy**:
- Start: λ_min = **200 req/s** (from std_mean)
- Stop: When **ρ = λ/(c·μ) ≥ 0.95** (near-saturation, avoid instability)
- Step: Δλ = **10 req/s**

**For each configuration**, we'll:
1. Vary λ from 200 to saturation point
2. Calculate χ = (1 - err) × λ
3. Use MMcK queueing model to compute W and L
4. Store all variable values for PyDASA analysis

In [ ]:
from src.queueing import Queue
import itertools
import pandas as pd

# STEP-A: Generate Configuration Grid
# ====================================
print("=" * 50)
print("STEP A: Generating Configuration Grid")
print("=" * 50)

# Define parameter ranges for grid search
# Error rates: 1% and 3% for two QoS agrements
err_values = [
    variables["err"].std_min, 
    variables["err"].std_max,
]

# Service rates: 400 and 500 req/s for two different services
mu_values = [
    variables["\\mu_{s}"].std_min,
    variables["\\mu_{s}"].std_max,
]

# Server counts: 1, 2, and 4 servers for redundancy/parallelism
c_values = [
    variables["c"].std_min,
    variables["c"].std_mean,
    variables["c"].std_max,
]          

# Generate all combinations (Cartesian product)
configs = list(itertools.product(err_values, mu_values, c_values))

print(f"\nGenerated {len(configs)} configurations:")
print(f"{'Config':<8} {'err':<8} {'μ (req/s)':<12} {'c (servers)':<12}")
print("-" * 50)
for i, (err, mu, c) in enumerate(configs, 1):
    print(f"{i:<8} {err:<8} {mu:<12} {c:<12}")

print(f"\n✓ Configuration grid ready: {len(configs)} configurations")
print("=" * 50)

# setting constants for te simulation
K_cst = variables["K"].std_setpoint          # Queue capacity [req]
B_cst = variables["B"].std_setpoint          # Buffer memory [bit]
rho_req_cst = variables["\\rho_{req}"].std_setpoint  # Data density [bit/req]
lambda_zero = variables["\\lambda_{s}"].std_setpoint  # Arrival rate [req/s]
lambda_step = variables["\\lambda_{s}"].step          # Arrival rate [req/s]

print("\nConstants for Simulation:")
print(f"\t- Queue Capacity (K): {K_cst} [req]")
print(f"\t- Buffer Memory (B): {B_cst} [bit]")
print(f"\t- Data Density (ρ_req): {rho_req_cst} [bit/req]")
print(f"\t- Arrival Rate (λ): {lambda_zero} [req/s]")
print(f"\t- Arrival Rate Step (Δλ): {lambda_step} [req/s]\n")
print("=" * 50)

# STEP B: DEFINE dataframe for data variables
print("=" * 50)
print("STEP B: Creating Queue Model Dataframe")
print("=" * 50)

q_cols = list(variables.keys())
data_df = pd.DataFrame(columns=q_cols)
print(f"Dataframe created with columns: {q_cols}")
print(f"Initial dataframe shape: {data_df.shape}")

# STEP C: Generate Data Points with MMcK Model
# =================================================
print("=" * 80)
print("STEP C: Generating Data with the M/M/c/K Queueing Model")
print("=" * 80)

rho_threshold = 0.95  # Stop when utilization reaches 95.0 %

# Storage for all data points
data_points = []

# Process each configuration
for config_idx, (err, mu, c) in enumerate(configs, 1):
    _msg = f"\t- Config: {config_idx}/{len(configs)}: "
    _msg += f"err={err}, μ={mu}, c={c}"
    print(_msg)
    lambda_t = lambda_zero
    config_data = []
    rho_t = 0.0
    i = 0
    while rho_t < rho_threshold:
        # Create and evaluate M/M/c/K queue model
        q = Queue(
            "M/M/s/K",
            _lambda=lambda_t,   # type: ignore
            mu=mu,              # type: ignore
            n_servers=c,        # type: ignore
            kapacity=K_cst      # type: ignore
    )
        q.calculate_metrics()
        chi_t = lambda_t * (1 - err)    # type: ignore

        # new data line
        data_t = [
            lambda_t,
            K_cst,
            rho_req_cst,
            q.avg_len,
            q.avg_wait,
            mu,
            chi_t,
            c,
            B_cst,
            err,
        ]
        
        data_df.loc[len(data_df)] = data_t

        rho_t = q.rho
        lambda_t = lambda_t + lambda_step   # type: ignore
        i += 1
    print(f"\t\tGenerated {i} data points for this configuration.")
        # print(f"\t\tλ={q._lambda:.1f} req/s | ρ={q.rho:.4e} | L={q.avg_len:.4e} req | W={q.avg_wait:.4e} s |")
        # config_data.append(q)

print(f"=== Generated {len(data_df)} data points!===")
print("\n" + "=" * 80)
print("DATA GENERATION SUMMARY")
print("=" * 80)
print(f"Total configurations: {len(configs)}")
print(f"Total data points: {len(data_df)}")
print(f"Average points per config: {len(data_df) / len(configs):.1f}")
print("=" * 80)

# Display sample data and statistics
print("=" * 100)
print("SAMPLE DATA POINTS (First 10 rows)")
print("=" * 100)
print(data_df.head(10).to_string(index=False))

print("\n" + "=" * 100)
print("DATA STATISTICS")
print("=" * 100)
print(data_df.describe())

# STEP 4: adding the data into the PyDASA Varibles

data = data_df.to_dict(orient="list")
for sym, var in variables.items():
    if sym in data:
        var.data = data[sym]
        msg = f"- Added data to '{sym}':({var.name}), len={len(var.data)}."
        print(msg)
    else:
        msg = f" - WARN: No data found for '{sym}' ({var.name}), leaving data empty."
        print(msg)

engine.variables = variables

STEP A: Generating Configuration Grid

Generated 12 configurations:
Config   err      μ (req/s)    c (servers) 
--------------------------------------------------
1        0.01     400.0        1.0         
2        0.01     400.0        2.0         
3        0.01     400.0        4.0         
4        0.01     500.0        1.0         
5        0.01     500.0        2.0         
6        0.01     500.0        4.0         
7        0.04     400.0        1.0         
8        0.04     400.0        2.0         
9        0.04     400.0        4.0         
10       0.04     500.0        1.0         
11       0.04     500.0        2.0         
12       0.04     500.0        4.0         

✓ Configuration grid ready: 12 configurations

Constants for Simulation:
	- Queue Capacity (K): 10.0 [req]
	- Buffer Memory (B): 128000000.0 [bit]
	- Data Density (ρ_req): 8000000 [bit/req]
	- Arrival Rate (λ): 200.0 [req/s]
	- Arrival Rate Step (Δλ): 10.0 [req/s]

STEP B: Creating Queue Model Dataframe
Dat

STEP 3: Creating Queue Model Dataframe
Dataframe created with columns: ['\\lambda_{s}', 'K', '\\rho_{req}', 'L', 'W', '\\mu_{s}', '\\chi_{s}', 'c', 'B', 'err']
Initial dataframe shape: (0, 10)


STEP 3: Generating Data with the M/M/c/K Queueing Model
	- Config: 1/12: err=0.01, μ=400.0, c=1.0
		Generated 19 data points for this configuration.
	- Config: 2/12: err=0.01, μ=400.0, c=2.0
		Generated 57 data points for this configuration.
	- Config: 3/12: err=0.01, μ=400.0, c=4.0
		Generated 133 data points for this configuration.
	- Config: 4/12: err=0.01, μ=500.0, c=1.0
		Generated 29 data points for this configuration.
	- Config: 5/12: err=0.01, μ=500.0, c=2.0
		Generated 76 data points for this configuration.
	- Config: 6/12: err=0.01, μ=500.0, c=4.0
		Generated 171 data points for this configuration.
	- Config: 7/12: err=0.04, μ=400.0, c=1.0
		Generated 19 data points for this configuration.
	- Config: 8/12: err=0.04, μ=400.0, c=2.0
		Generated 57 data points for this configuration.
	- Config: 9/12: err=0.04, μ=400.0, c=4.0
		Generated 133 data points for this configuration.
	- Config: 10/12: err=0.04, μ=500.0, c=1.0
		Generated 29 data points for this configuration.
	- Config:

SAMPLE DATA POINTS (First 10 rows)
 \lambda_{s}    K  \rho_{req}        L        W  \mu_{s}  \chi_{s}   c           B  err
       200.0 10.0   8000000.0 0.994626 0.004976    400.0     198.0 1.0 128000000.0 0.01
       210.0 10.0   8000000.0 1.096069 0.005223    400.0     207.9 1.0 128000000.0 0.01
       220.0 10.0   8000000.0 1.206876 0.005492    400.0     217.8 1.0 128000000.0 0.01
       230.0 10.0   8000000.0 1.327896 0.005783    400.0     227.7 1.0 128000000.0 0.01
       240.0 10.0   8000000.0 1.459947 0.006098    400.0     237.6 1.0 128000000.0 0.01
       250.0 10.0   8000000.0 1.603781 0.006437    400.0     247.5 1.0 128000000.0 0.01
       260.0 10.0   8000000.0 1.760034 0.006802    400.0     257.4 1.0 128000000.0 0.01
       270.0 10.0   8000000.0 1.929173 0.007192    400.0     267.3 1.0 128000000.0 0.01
       280.0 10.0   8000000.0 2.111440 0.007607    400.0     277.2 1.0 128000000.0 0.01
       290.0 10.0   8000000.0 2.306806 0.008046    400.0     287.1 1.0 128000000.0 0.

In [ ]:
data_df

✓ Added data to '\lambda_{s}':(Arrival Rate), len=970
✓ Added data to 'K':(Queue Capacity), len=970
✓ Added data to '\rho_{req}':(Data Density), len=970
✓ Added data to 'L':(Queue Length), len=970
✓ Added data to 'W':(Waiting Time), len=970
✓ Added data to '\mu_{s}':(Service Rate), len=970
✓ Added data to '\chi_{s}':(Response Rate), len=970
✓ Added data to 'c':(Servers), len=970
✓ Added data to 'B':(Buffer Memory), len=970
✓ Added data to 'err':(Error Rate), len=970


In [ ]:
# Create PyDASA MonteCarloSimulation with grid data
from pydasa.workflows.practical import MonteCarloSimulation

print("\n" + "=" * 80)
print("Creating PyDASA MonteCarloSimulation with Grid Data")
print("=" * 80)

# Create Monte Carlo handler
mc_grid = MonteCarloSimulation(
    _idx=0,
    _fwk="CUSTOM",
    _schema=schema,
    _name="Grid-Based Queue Analysis",
    _cat="DATA",  # Custom category for grid-based sampling
    _experiments=len(data_df),
    _variables=engine.variables,
    _coefficients=engine.coefficients
)

print(f"✓ MonteCarloSimulation created")
print(f"\t- Framework: {mc_grid.fwk}")
print(f"\t- Experiments: {mc_grid.experiments}")
print(f"\t- Variables: {len(mc_grid._variables)}")
print(f"\t- Coefficients: {len(mc_grid._coefficients)}")
print("=" * 80)


Creating PyDASA MonteCarloSimulation with Grid Data
✓ MonteCarloSimulation created
  - Framework: CUSTOM
  - Experiments: 970
  - Variables: 10
  - Coefficients: 10


### M/M/c/K System Diagram

```
                    ┌─────────────────────────────────────────────────────┐
                    │  M/M/c/K System (Capacity = K)                      │
                    │                                                     │
  Arrivals (λ)      │     Queue          Servers (μ)      Response (χ)   │
  [S/T]             │   ┌─┬─┬─┐                                          │
                    │   │ │ │ │                                          │
    ● ● ●  ─────────┼──→│ │ │ │─────→  Server 1 (μ) ──────────────────→ ●  Served
    ● ● ●           │   └─┴─┴─┘    ╱                                    │  Requests
    ● ● ●           │    (K-c)     ╱                                     │  χ = (1-err)·λ
  (Poisson)         │   slots    ●─────→  Server 2 (μ) ─────────────→ ● │  [S/T]
                    │           ╲                                        │
  When FULL:        │            ╲                                       │
    ● ● ●  ─────X   │             ────→  Server c (μ) ────────────────→ ●│
                    │                                                     │
  Blocked/          │  ↑____________________________________________↑    │
  Rejected          │         K = Total Capacity (queue + servers)       │
                    │                                                     │
  Error Rate (err)  │  Note: χ = (1 - err) · λ                          │
  Affects Response  │  Some requests fail → reduces effective throughput │
  [dimensionless]   │                                                     │
                    └─────────────────────────────────────────────────────┘
                    
  Legend:
  - λ (lambda): Arrival rate [S/T]
  - μ (mu): Service rate per server [S/T]
  - c: Number of parallel servers [S]
  - K: Total capacity (queue + servers) [S]
  - χ (chi): Effective response rate = (1 - err) · λ [S/T]
  - err: Error/failure rate [dimensionless, 0 to 1]
```